# Starting and testing OVRO-LWA

### Setup

1. Use dashboard to confirm that services are up for etcd, bifrost pipelines, and data capture.
2. Identify server names for snap2, gpu, and data capture.
3. Set list of server names in notebook.
4. Execute cells


### F-eng
* Repo https://github.com/realtimeradio/caltech-lwa
* python install "control_sw"

### X-eng
* Repo https://github.com/realtimeradio/caltech-bifrost-dsp
* python install "pipeline-control"

### mnc
* Repo https://github.com/ovro-lwa/mnc_python
* python install "mnc_python"

In [4]:
snap2names = ['snap01']  # f-eng
feng_config = '/home/ubuntu/proj/lwa-shell/caltech-lwa/control_sw/config/lwa_corr_config.yaml'  # f-eng and x-eng
xhosts = ['lxdlwagpu02']  # x-eng
npipeline = 4  # x-eng
drips = ['10.41.0.25', '10.41.0.41']  # data recorder (two ips for lxdlwagpu09)
dest_ip = list(sorted(drips*2))  # correlator -> data recorder (destination IPs)
dest_port = [10001+i//npipeline for i in range(npipeline*len(xhosts))]  # correlator -> data recorder (destination ports)
drroot = [f'drvs{ip.split(".")[-1]}' for ip in drips]  # data recorder root names
drids = [f'{drr}{dp-10000:02}' for dp in set(dest_port) for drr in drroot]  # data recorder ids

In [5]:
print(snap2names, xhosts, drips, dest_ip, dest_port, drids)

['snap01'] ['lxdlwagpu02'] ['10.41.0.25', '10.41.0.41'] ['10.41.0.25', '10.41.0.25', '10.41.0.41', '10.41.0.41'] [10001, 10001, 10001, 10001] ['drvs2501', 'drvs4101']


In [6]:
from lwa_f import snap2_fengine
from lwa352_pipeline_control import Lwa352CorrelatorControl
from mnc import mcs, common, ezdr

In [14]:
for snap2name in snap2names:
    lwa_f = snap2_fengine.Snap2Fengine(snap2name)
# if there are weird errors, probably need to program. fpgfile is optional argument.
#    lwa_f.program(fpgfile="/home/ubuntu/proj/lwa-shell/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2022-01-10_2153.fpg")

    lwa_f.cold_start_from_config(feng_config)

2022-03-30 11:57:56,651 - lwa_f.blocks.block:snap01 - WARNING -  - hostname (10.40.0.157) couldn't be turned into integer serial
2022-03-30 11:57:56,929 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-03-30 11:57:57,206 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1
2022-03-30 11:57:57,220 - lwa_f.snap2_fengine:snap01 - INFO - Trying to configure output with config file /home/ubuntu/proj/lwa-shell/caltech-lwa/control_sw/config/lwa_corr_config.yaml
2022-03-30 11:57:57,223 - lwa_f.snap2_fengine:snap01 - INFO - Loading existing firmware to snap01
File in flash is:  snap2_f_200msps_64i_4096c_2022-01-10_2153.fpg
2022-03-30 11:58:17,380 - lwa_f.snap2_fengine:snap01 - INFO - Loaded snap2_f_200msps_64i_4096c_2022-01-10_2153.fpg
2022-03-30 11:58:17,697 - lwa_f.blocks.block:snap01 - WARNING -  - hostname (10.40.0.157) couldn't be turned into integer serial
2022-03-30 11:58:17,913 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC

WARNING >>>> SPI readback error (FMC 1, chip 0, addr 0xf)
WARNING >>>> SPI readback error (FMC 1, chip 0, addr 0x7)
WARNING >>>> SPI readback error (FMC 1, chip 0, addr 0x24)
WARNING >>>> SPI readback error (FMC 1, chip 0, addr 0x46)
WARNING >>>> SPI readback error (FMC 1, chip 0, addr 0x40)
WARNING >>>> SPI readback error (FMC 1, chip 0, addr 0x25)
WARNING >>>> SPI readback error (FMC 1, chip 1, addr 0xf)
WARNING >>>> SPI readback error (FMC 1, chip 1, addr 0x7)
WARNING >>>> SPI readback error (FMC 1, chip 1, addr 0x24)
WARNING >>>> SPI readback error (FMC 1, chip 1, addr 0x46)
WARNING >>>> SPI readback error (FMC 1, chip 1, addr 0x40)
WARNING >>>> SPI readback error (FMC 1, chip 1, addr 0x25)
WARNING >>>> SPI readback error (FMC 1, chip 2, addr 0xf)
WARNING >>>> SPI readback error (FMC 1, chip 2, addr 0x7)
WARNING >>>> SPI readback error (FMC 1, chip 2, addr 0x24)
WARNING >>>> SPI readback error (FMC 1, chip 2, addr 0x46)
WARNING >>>> SPI readback error (FMC 1, chip 2, addr 0x40)
WAR

2022-03-30 11:58:26,252 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 0 clock rates: [490705694, 98121320, 98122738, 0, 0]
2022-03-30 11:58:26,253 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 0: MMCM locked
2022-03-30 11:58:27,261 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 1 clock rates: [491076821, 98214891, 98213947, 98240897, 0]
2022-03-30 11:58:28,267 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 board 0 clock rates: [7446, 745, 695, 0, 0]
2022-03-30 11:58:28,268 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 board 0: MMCM locked
2022-03-30 11:58:29,274 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 board 1 clock rates: [490636569, 98157513, 98155153, 98157040, 0]
2022-03-30 11:58:30,242 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 Scanning data delays


100% (64 of 64) |########################| Elapsed Time: 0:00:06 Time:  0:00:06


2022-03-30 11:58:37,184 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane delays:
[[312, 304, 312, 296, 304, 296, 296, 296], [272, 264, 256, 256, 256, 264, 248, 256], [248, 248, 248, 248, 248, 248, 248, 256], [280, 288, 272, 280, 280, 280, 280, 272], [256, 256, 256, 264, 264, 256, 256, 256], [232, 240, 240, 240, 232, 232, 232, 232], [248, 232, 232, 240, 232, 248, 232, 232], [272, 272, 272, 272, 272, 288, 288, 264]]
2022-03-30 11:58:37,184 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane slacks:
[[40, 40, 40, 40, 40, 32, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 32], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 32], [40, 40, 32, 40, 40, 40, 40, 40], [40, 40, 40, 32, 40, 32, 40, 40], [32, 40, 32, 32, 40, 40, 40, 32]]
2022-03-30 11:58:37,185 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXX
2022-03-30 11:58:37,185 - lwa_f.blocks.block:snap01 - INF

...

100% (64 of 64) |########################| Elapsed Time: 0:00:06 Time:  0:00:06


2022-03-30 11:59:34,073 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 0 because nowhere was good
2022-03-30 11:59:34,084 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 1 because nowhere was good
2022-03-30 11:59:34,093 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 2 because nowhere was good
2022-03-30 11:59:34,103 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 3 because nowhere was good
2022-03-30 11:59:35,031 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 Scanning data delays


100% (64 of 64) |########################| Elapsed Time: 0:00:06 Time:  0:00:06


2022-03-30 11:59:41,899 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane delays:
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [280, 288, 280, 296, 288, 288, 280, 288], [224, 224, 216, 216, 224, 216, 232, 216], [272, 264, 264, 280, 264, 264, 264, 264], [304, 304, 304, 296, 296, 296, 304, 296]]
2022-03-30 11:59:41,899 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane slacks:
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [40, 40, 40, 40, 40, 40, 32, 40], [40, 40, 32, 32, 40, 32, 40, 32], [40, 32, 32, 40, 32, 32, 32, 32], [40, 40, 40, 32, 32, 32, 40, 32]]
2022-03-30 11:59:41,900 - lwa_f.blocks.block:snap01 - WARNING - adc - Delay solutions have small slack
2022-03-30 11:59:41,901 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    |XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-03-30 11:59:41,901 - lwa_f.blocks.block:snap0

2022-03-30 11:59:41,922 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 1:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|---XXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-03-30 11:59:41,922 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 2:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-03-30 11:59:41,923 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 3:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXX
2022-03-30 11:59:41,923 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 4:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-03-30 11:59:41,923 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 5:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|---XXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-03-30 11:59:41,924 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 6:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|---XXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-03-30 11:59:41,924 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 7:    XXXXXXXX

In [5]:
lwa_corr = Lwa352CorrelatorControl(xhosts, npipeline_per_host=npipeline)
lwa_corr.stop_pipelines()   # maybe not needed?
lwa_corr.start_pipelines() 
lwa_corr.configure_corr(dest_ip=dest_ip, dest_port=dest_port)

2021-09-29 23:11:26 [WARNING ] Timeout waiting for pipelines to come up after 60.0 seconds
2021-09-29 23:11:26 [INFO    ] Setting max data output rate per pipeline to 5000.0 Mbit/s
2021-09-29 23:11:26 [INFO    ] Setting pipeline lxdlwagpu02:0 data destination to 10.41.0.25:10001
2021-09-29 23:11:46 [INFO    ] Setting pipeline lxdlwagpu02:1 data destination to 10.41.0.25:10001
2021-09-29 23:11:56 [INFO    ] Setting pipeline lxdlwagpu02:2 data destination to 10.41.0.41:10001
2021-09-29 23:11:56 [INFO    ] Setting pipeline lxdlwagpu02:3 data destination to 10.41.0.41:10001
2021-09-29 23:11:56 [INFO    ] Arming correlator core
2021-09-29 23:11:57 [INFO    ] All pipelines in 'waiting' state as expected
2021-09-29 23:11:57 [INFO    ] Waiting 10 seconds for trigger
2021-09-29 23:12:07 [INFO    ] All pipelines in 'running' state as expected
2021-09-29 23:12:07 [INFO    ] Arming correlator accumulator core
2021-09-29 23:12:08 [INFO    ] All pipelines in 'waiting' state as expected
2021-09-29 23

In [6]:
rs = ezdr.Lwa352RecorderControl('slow')
rs.start()

Found 0 slow recorders: 


[]

### Read data and visualize/test

## Stop

In [9]:
rs.stop()
lwa_corr.stop_pipelines()